# Klasifikacija zvezd ~ projekt pri predmetu Tehnologije znanja

Spektralna razvrstitev zvezd je v astronomiji razvrstitev zvezd po njihovih spektralnih značilnostih (med njimi spadajo: 
-> fotosferska temperatura
-> črte elementov, ki jih absorbirajo
-> ...
).
Posledično so zvezde razdeljene še v podrazrede.

Elektromagnetno sevanje analiziramo tako, da ga s prizmo ali uklonsko mrežico razdelimo v spekter, ki prikazuje mavrico barv, prepredeno s spektralnimi črtami.
~> Vsaka črta označuje določen kemijski element oz. molekulo, pri čemer moč črte označuje številčnost tega elementa.
Moči različnih spektralnih linij se razlikujejo predvsem zaradi temperature fotosfere, pri čemer 

## Uvoz knjižnic ter osnovne datoteke

Preden se lotimo programiranja učnega modela, moramo uvoziti vse potrebne knjižnice, ki nam bodo pomagale pri obdelavi, učenju ter prikazovanju podatkov.

In [44]:
import numpy as np # matematična knjižnica za manipulacijo s podatki
import pandas as pd # knjižnica za podatkovno analizo
import matplotlib.pyplot as plt # knjižnica za prikaz podatkov v grafični obliki
import seaborn as sns # knjižnica za prikaz podatkov v grafični obliki
import plotly.express as px # knjižnica za prikaz podatkov v grafični obliki

from sklearn.model_selection import train_test_split # metoda za razdelitev podatkovne množice na učno mn. in testno mn.

In [18]:
df = pd.read_csv('zvezde.csv')
df.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M


## Priprava ter pregled podatkov

Preden se bomo lotili priprave podatkov, bomo prvo pogledali, kako naša podatkovna množica sploh izgleda (oziroma kakšne podatke vsebuje).

In [68]:
# Razmerje tipov zvezd v podatkovni množici
labels = ['Brown dwarf', 'Red dwarf', 'White dwarf', 'Main sequence', 'Supergiant', 'Hypergiant']

graf = px.pie(df['Star type'].value_counts(), values='Star type', names=labels,hole=0.4,opacity=0.6, title = 'Star Type')
graf.update_traces(textposition='outside', textinfo='percent+label')
graf.show()

Presenetljivo je to, da ima podatkovna množica enako število vsakega tipa zvezd.

In [47]:
# Prikaz znač
stolpci = ['Temperature (K)', 'Luminosity(L/Lo)', 'Radius(R/Ro)',
       'Absolute magnitude(Mv)','Star color']

for stolpec in stolpci:
    #plt.figure()
    #plt.tight_layout()
    
    #graf = sns.scatterplot(x=stolpec, y='Star type', data=df, hue='Star type')
    
    fig = px.scatter(df, x = stolpec, color = 'Star type')
    fig.show()


Glede na prikazane podatke lahko na priv pogled za vsako kategorijo zvezd sklepamo o naslednjih značilnostih (radij je zaradi trivialnosti izključen):


|  **Ugotovitve** | Temperatura      | Svetilnost | Absolutni izsev | Barva |
|-----------------|------------------|------------|-----------------|-------|
| Rjave pritlikavke | Nizka temperatura | Nizka svetilnost | Visok absolutni izsev | Rdečkasto-rjavkaste barve |
| Rdeče pritlikavke | Nizka temperatura | Nizka svetilnost | Visok absolutni izsev | Rdeče barve |
| Bele pritlikavke  | Nizka oz. srednje visoka temperatura | Nizka svetilnost | Visok absolutni izsev | Odtenki bele barve, modra |
| Supervelikanke | Večinoma srednje visoka temperatura, tudi visoka | Srednje visoka ter visoka svetilnost | Nizek absolutni izsev | Modre barve |
| Hipervelikanke |  Večinoma nizka temperatura, tudi visoka | Večinoma visoka svetilnost | Zelo nizek absolutni izsev | Rdeče, oranžne, bele, modre barve |

In [26]:
# Tipi podatkov v podatkovni množici
df.dtypes

Temperature (K)             int64
Luminosity(L/Lo)          float64
Radius(R/Ro)              float64
Absolute magnitude(Mv)    float64
Star type                   int64
Star color                 object
Spectral Class             object
dtype: object

 $ \uparrow $ V podatkovni množici imamo večinoma številske podatke, dve vrstici sta kategorični.

In [27]:
# Število manjkajočih vrednosti po posameznem stolpcu v podatkovni množici
df.isna().sum()

Temperature (K)           0
Luminosity(L/Lo)          0
Radius(R/Ro)              0
Absolute magnitude(Mv)    0
Star type                 0
Star color                0
Spectral Class            0
dtype: int64

Prednost te podatkovne množice je, da ne vsebuje <u>nobenih ničelnih podatkov</u>. To pomeni, da ne potrebujemo odstranjevati manjkajočih vrednosti, tako da se lahko lotimo obdelave podatkov.

Prvo razdelimo podatkovno na **učno množico** ter na **testno množico**.
-> *Model bomo naučili z učno množico in nato s pomočjo testne množice model stestirali in pogledali, kako uspešen je bil naš model pri klasificiranju zvezd*

Pri tej podatkovni množici bomo napovedovali tip zvezde -> **Star type**. 

Učna množica bo vsebovala naslednje podatke:
* **Temperature [K]** -> temperatura zvezde, merjena v Kelvinski stopinjski lestvici
* **Luminosity  [$ \frac{L}{L_{\odot\ }} $]** -> svetilnost/izsev zvezde v razmerju s svetilnostjo Sonca
    *  _Svetilnost Sonca_ $ \rightarrow  L_{\odot\ }=3.83\cdot 10^{26}W $
* **Relative Radius [ $  \frac{r}{r_{\odot\ }} $]** -> razmerje polmera zvezde v primerjavi s polmerom Sonca
    *  _Polmer sonca_ $ \rightarrow  r_{\odot\ }=6.960\cdot 10^{8}m $
* **Absolute magnitude [Mv]** -> absolutni izsev zvezde
* **Star color [blue, white, yellow, yellow-orange, red ...]** -> barva zvezde
* **Spectral class [O, B, A, F, G, K, M]** -> spektralni razred zvezde, kateremu pripada

Testna množica vsebuje:
* **Star type [0, 1, 2, 3, 4, 5]** -> tip zvezde
    * 0 -> Rjava pritlikavka _(Brown dwarf)_
    * 1 -> Rdeča pritlikavka _(Red dwarf)_
    * 2 -> Bela pritlikavka _(White dwarf)_
    * 3 -> Main sequence (pritlikavka) _(Dwarf)_ 
    * 4 -> Supervelikanka _(Supergiant)_
    * 5 -> Hipervelikanka _(Hypergiant)_

In [23]:
test = df['Star type']
podatki = df.drop(['Star type'], axis=1)

V tej podatkovni množici imamo poleg številskih podatkov tudi **kategorične vrednosti**. Za učenje našega podatkovnega modela moramo v naši podatkovni množici imeti <u>le številske vrednosti</u>, zato moramo vse kategorične vrednosti zamenjati z ustreznimi številskimi. Vendar to je nepraktično delati na roke. Kako se lotimo te zadeve?

Pythonova knjižnica pandas vsebuje metodo prav zato -> **get_dummies()**. Ta metoda nam bo vse kategorične vrednosti spremenila v številske tako, da bo vsaka posamična kategorija pridobila svoj stolpec v dataframe-u in tam označila, če je nek vnos tip te kategorije ali ne
* 1 -> je ta tip kategorije
* 0 -> ni ta tip kategorije

In [24]:
podatki = pd.get_dummies(podatki)

podatki.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star color_Blue,Star color_Blue,Star color_Blue White,Star color_Blue white,Star color_Blue white,Star color_Blue-White,...,Star color_white,Star color_yellow-white,Star color_yellowish,Spectral Class_A,Spectral Class_B,Spectral Class_F,Spectral Class_G,Spectral Class_K,Spectral Class_M,Spectral Class_O
0,3068,0.002400,0.1700,16.12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3042,0.000500,0.1542,16.60,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2600,0.000300,0.1020,18.70,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2800,0.000200,0.1600,16.65,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1939,0.000138,0.1030,20.06,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


Preden začnemo implementirati model ter ga učiti, moramo našo učno ter testno množico vsako razbiti v učno ter testno množico.

Razdelitev bo potekala na naslednji način:
* Učna ter testna množica bosta razdeljeni v naslednjem razmerju:
    * Učna množica -> 70%
    * Testna množica -> 30%
* Za **Random state** vrednost bom uporabil 123

In [22]:
x_train, x_test, y_train, y_test = train_test_split(podatki, test, test_size=0.3, random_state=123)

## Izbira ter učenje modela